# Practical 6a Classification using SVM

So far we have just been using the datasets provided with sklearn. However, we normally need to use datasets from other locations. So the first part of this practical will give an introduction on how to read in CVS files from a file or URL.

Once you've done this you will need to process the data so that missing values are removed or replaced with sensible values. 

Then finally we can get down to the machine learning. The way you use SVM is very similar to what you did with Logistic Regression - so we don't need to give you all the details for that part. You can just look back at that.

The data: We're going to use the titanic dataset. This dataset contains various features such as the sex of a passenger and the class they were travelling in. The idea is to see if we can predict if the passanger is likely to have survived.

## Reading in the data

Pandas nicely provides you with a method to read in data from a CSV file. The file can either be on your local hard disk or at a URL.

Once you've read in the data the first thing to do is to have a quick look at it - print it out.

In [ ]:
import pandas as pd

data = pd.read_csv("http://homepages.cs.ncl.ac.uk/stephen.mcgough/data/titanic.csv")

print(data)

## What are the different features

We can list the features using the list command

The features are:

| feature | description |
|-----|------|
| pclass | The class the passenger was travelling in |
| survived | Whether the passenger survived or not |
| name | Name of the passenger |
| sex | Gender of the passenger |
| age | Age of the passenger |
| sibsp | Number of siblings or spouses onboard |
| parch | Number of parents or children onboard |
| ticket | Ticket number |
| fare | How much they payed for the ticket |
| cabin | Cabbin number |
| embarked | Where they embarked from (C = Cherbourg; Q = Queenstown; S = Southampton) |
| boat | Lifeboat they got onto |
| body | Unknown |
| home.dest | Where they were eventually travelling to |


In [ ]:
list(data)

We now need to look at each of the features and decide which ones to keep and which ones to delete. You can use the value_counts() method to see how many unique values there are. Below is the one for 'age'.

## Exercise
Try each of the different features and see how many unique values there are and how much missing data there is (recorded here with '?').

In [ ]:
data['age'].value_counts()

Removing features that either have unique values for all or too much missing data.

We're going to remove the features 'name' and 'ticket' as almost all passengers have a unique name/ticket - so this is unlikely to be of help.

We're going to remove 'cabin', 'boat', 'body' and 'home.dest' as most of these values are missing.

To remove a feature we drop the appropriate column from the pandas dataframe.

In [ ]:
dataClean = data.drop(columns=['name', 'cabin', 'ticket', 'boat', 'body', 'home.dest'])

We had two passengers who don't have a record of where they embarked from. We need to set a value for these - let's assume it is Southampton (S). We can come back to this later and try the other options to see if they give us better results.

In [ ]:
dataClean['embarked'].replace(['?'], ['S'], inplace=True)

There are a number of passengers missing an age. For now let's assume that they have the median age of all the other passangers. We can again come back to this later and try other estimates.

In [ ]:
withoutMissingAge = dataClean.drop(dataClean[dataClean.age=='?'].index)
medianAge = withoutMissingAge['age'].median()
dataClean['age'].replace(['?'], [medianAge], inplace=True)

In the same way there are a number of passangers who don't have a fare value for their travels. We'll again assume that the median is a good estimate. We can come back later and try other estimates.

## Exercise
Using the example of replacing missing ages above replace the missing fares with the median value.

The data should now be clean. We should look at all of the remaining features to check that there are no more missing values.

## Exercise
Check each of the features for missing values. You can do this using values_counts().

In [ ]:
dataClean['age'].value_counts()

The feature 'sex' is a catagorical feature and needs to be converted into a One Hot Encoded set of features. First we use get_dummies() to create a One Hot Encoded version of the feature. But this looses all the other features so we need to join it back to the original data and delete the original feature.

In [ ]:
sexOHE = pd.get_dummies(dataClean['sex'], prefix='sex') # prefix is the text to start the new feature names with. The rest of the 
                                                        # feature name will be the catagorical value. So in this case we get
                                                        # sex_female or sex_male
dataClean = dataClean.join(sexOHE)
dataClean = dataClean.drop(columns=['sex'])

We need to do the same for the other catagorical feature - embarked. 

## Exercise
Produce a One Hot encoding for embarked in the same way as sex was done above.

## Producing the X and y data

The X data is all of the features without the variable we want to predict and the y data is just the variable we want to predict. 

We can remove the variable we want to predict (survived) from the data to produce X.

We can keep just the variable we want to predict using filter.

In [ ]:
X = dataClean.drop(columns=['survived'])
y = dataClean.filter(['survived']).values.ravel() # we need a 1D array for y - hence the need for values.ravel() to reshape the data 

## Split the data

We can now split the data into training and test data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Now the machine learning

sklearn is really good in the sense that all of the machine learning processes work in prety much the same way:

```
# import the appropriate classifier
from sklearn.linear_model import LogisticRegression

# create an instance of that classifier
logistic = LogisticRegression()

# Train it on our data
logistic.fit(X_train, y_train)
```

So we just need to change sklearn.linear_model.LogisticRegression to sklearn.svm.SVC. Then change the instance you create to the correct classifier.

## Exercises
1. Create a Support Vector Classifer for the titanic data.
2. For your classifier look at the accuracy, precision, recall, F1 and confusion matrix.
3. Look at changing the C value and the kernel for SVC to see if you can improve the results.

# Write your code from here